In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False ## 마이너스 부호 깨짐 방지

pd.set_option('display.max_columns', None)                       # 출력할 열의 최대개수
pd.set_option('display.max_colwidth', None)                      # 출력할 열의 너비
# pd.reset_option('display.max_rows', None)                    # 출력할 행의 최대개수  None
pd.set_option('display.unicode.east_asian_width', True) 

### 컬럼 설명
### train_users_2.csv
- id : 사용자 아이디
- date_account_created: 계정 생성 날짜
- timestamp_first_active: 첫 번째 활동의 타임스탬프입니다. 사용자가 가입하기 전에 검색할 수 있으므로 date_account_created 또는 date_first_booking보다 이전일 수 있습니다.
- date_first_booking: 첫 예약 날짜
- 성별
- 나이
- signup_method - 가입_방법
- signup_flow: 사용자가 가입하기 위해 온 페이지
- language: 국제 언어 기본 설정
- affili_channel: 유료 마케팅의 종류
- affiliate_provider: 마케팅이 이루어지는 곳. 구글, 크레이그리스트, 기타
- first_affiliate_tracked: 가입하기 전에 사용자가 처음으로 상호작용한 마케팅은 무엇입니까?
- signup_app - 가입_앱
- first_device_type - 첫 번째_장치_유형
- first_browser - 첫 번째_브라우저
- country_destination: 예측할 대상 변수입니다.
### session.csv - 사용자에 대한 웹 세션 로그
- user_id: users 테이블의 'id' 컬럼과 조인됩니다.
- action - 행동 
- action_type
- action_detail - 액션_디테일
- device_type - 기기 종류
- secs_elapsed - 초_경과 
#### country.csv - 이 데이터세트의 목적지 국가와 해당 위치에 대한 요약 통계
#### age_gender_bkts.csv - 사용자의 연령대, 성별, 목적지 국가에 대한 요약 통계
#### Sample_submission.csv - 예측 제출을 위한 올바른 형식

In [4]:
## sessions
sessions = pd.read_csv('sessions.csv')
sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [5]:
sessions['user_flow'] = sessions['action'].astype('str') + '+' + sessions['action_type'].astype('str') + '+' + sessions['action_detail'].astype('str')

In [6]:
## train_users_2
train = pd.read_csv('train_users_2.csv')
train.rename(columns={'id':'user_id'},inplace=True)
train.head()

,user_id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [7]:
train['timestamp_first_active'] = pd.to_datetime(train['timestamp_first_active'], format = '%Y%m%d%H%M%S').dt.strftime('%Y-%m-%d')
train['date_account_created'] = pd.to_datetime(train['date_account_created'])
train['date_first_booking'] = pd.to_datetime(train['date_first_booking'], errors='coerce')

In [8]:
train.isnull().sum()

user_id                         0
date_account_created            0
timestamp_first_active          0
date_first_booking         124543
gender                          0
age                         87990
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6065
signup_app                      0
first_device_type               0
first_browser                   0
country_destination             0
dtype: int64

### Train_Session 병합

In [9]:
df = pd.merge(sessions, train, on='user_id', how='inner')
df

,user_id,action,action_type,action_detail,device_type,secs_elapsed,user_flow,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0,lookup+nan+nan,2014-01-01,2014-01-01,2014-01-04,MALE,62.0,basic,0,en,sem-non-brand,google,omg,Web,Windows Desktop,Chrome,other
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0,search_results+click+view_search_results,2014-01-01,2014-01-01,2014-01-04,MALE,62.0,basic,0,en,sem-non-brand,google,omg,Web,Windows Desktop,Chrome,other
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0,lookup+nan+nan,2014-01-01,2014-01-01,2014-01-04,MALE,62.0,basic,0,en,sem-non-brand,google,omg,Web,Windows Desktop,Chrome,other
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0,search_results+click+view_search_results,2014-01-01,2014-01-01,2014-01-04,MALE,62.0,basic,0,en,sem-non-brand,google,omg,Web,Windows Desktop,Chrome,other
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0,lookup+nan+nan,2014-01-01,2014-01-01,2014-01-04,MALE,62.0,basic,0,en,sem-non-brand,google,omg,Web,Windows Desktop,Chrome,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5537952,nw9fwlyb5f,index,data,reservations,iPhone,245.0,index+data+reservations,2014-06-30,2014-06-30,NaT,-unknown-,NaN,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,NDF
5537953,nw9fwlyb5f,unavailabilities,data,unavailable_dates,iPhone,286.0,unavailabilities+data+unavailable_dates,2014-06-30,2014-06-30,NaT,-unknown-,NaN,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,NDF
5537954,nw9fwlyb5f,notifications,submit,notifications,iPhone,830.0,notifications+submit+notifications,2014-06-30,2014-06-30,NaT,-unknown-,NaN,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,NDF
5537955,nw9fwlyb5f,search,click,view_search_results,iPhone,101961.0,search+click+view_search_results,2014-06-30,2014-06-30,NaT,-unknown-,NaN,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,NDF


# 예약 (1)  vs 비예약 (0) 구분

In [10]:
df['binary_target'] = df['country_destination'].replace({'NDF': 0}, inplace=False)
df.loc[df['binary_target'] != 0, 'binary_target'] = 1

# 가입하지 않고 예약한 자 확인
- 로그 기록된 인원은 예약을 먼저하고 가입한 인원이 존재하지 않는다. 따라서 세션 데이터 유저는 가입 후 행동이 기록된다. 

In [11]:
df['booking_creation_delay'] = df['date_first_booking'] - df['date_account_created']

In [12]:
df[df['booking_creation_delay'] < pd.Timedelta(0)]

,user_id,action,action_type,action_detail,device_type,secs_elapsed,user_flow,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,binary_target,booking_creation_delay


## action 계층 분석 

In [13]:
df['user_flow']

0                                    lookup+nan+nan
1          search_results+click+view_search_results
2                                    lookup+nan+nan
3          search_results+click+view_search_results
4                                    lookup+nan+nan
                             ...                   
5537952                     index+data+reservations
5537953     unavailabilities+data+unavailable_dates
5537954          notifications+submit+notifications
5537955            search+click+view_search_results
5537956            search+click+view_search_results
Name: user_flow, Length: 5537957, dtype: object

In [22]:
df[ (df['action_type'].isnull()) & (df['action_detail'].isnull()) ]['action'].unique()


array(['lookup', 'show', 'track_page_view', 'widget', 'uptodate', 'index',
       'campaigns', 'currencies', 'localization_settings', 'update',
       'phone_verification', 'satisfy', 'track_activity', 'check',
       'signed_out_modal', 'disaster_action', 'similar_listings_v2'],
      dtype=object)